In [1]:
## Parameters specific to where your folders are and your data
parameter_file = '../params/params.yaml'
import yaml
import sys

with open(parameter_file,'r') as f:
    doc = yaml.load(f)

#p = dic2obj(**doc)

data_folder = doc['data_folder']
tissues = doc['tissues'].split(',')
sys.path.append(doc['pipeline_path'])
ref_fa = doc['ref_fa']
anno_gff=doc['annotation']
mRNA_peak_file = doc["mRNA_peak_file"]

tss_annotation = doc['tss_annotation']


import os
from os.path import join
import sys
import pandas as pd
import matplotlib
import seaborn as sns
import pickle
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from itertools import product
import glob
import re
from matplotlib_venn import venn2
from matplotlib import rcParams
import inspect
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sys.setrecursionlimit(3000)
%load_ext autoreload
%autoreload 2
rcParams['figure.figsize'] = 8, 6
from tqdm import *
from os.path import basename
##mpl.use('Agg')
#mpl.style.use('ggplot')
#mpl.style.use('fivethirtyeight')
from Homer import *
import helper
import create_output
print('Number of tissues: ',len(tissues))

from collections import Counter


('Number of tissues: ', 13)


## File and folder names. 
(input is bam files, and design csv file)

In [6]:
chip_dir = os.path.join(data_folder,"external", "Sequencing", "PRJEB9291","processed")
design = pd.read_csv(os.path.join(chip_dir, "design.csv"))
bam_dir = os.path.join(chip_dir,"sortBam")
tag_dir = os.path.join(chip_dir, "tagDir")
peak_dir = os.path.join(chip_dir, "peaksDir")
bedgraph_dir = os.path.join(chip_dir, "bedgraphs")
if not os.path.exists(tag_dir):
    os.mkdir(tag_dir)
if not os.path.exists(peak_dir):
    os.mkdir(peak_dir)
if not os.path.exists(bedgraph_dir):
    os.mkdir(bedgraph_dir)
bam_files = glob.glob(bam_dir + "/*bam")

In [4]:
for bam_in in bam_files: 
    tag_name = bam_in.split("trim_")[1].split(".bam")[0]
    full_dir = os.path.join(chip_dir, tag_dir, tag_name)
    make_tag_directory(bam_in, full_dir, ref_fa)
    #cmd = "makeTagDirectory {full_dir} {bam_in}".format(full_dir=full_dir, bam_in=bam_in)
    #os.system(cmd)

makeTagDirectory /data/isshamie/TSS/NCBI_PICR_103_Processed/external/Sequencing/PRJEB9291/processed/tagDir/ERS2502088_1 -genome /data/isshamie/genome/ncbi_anno_103/GCF_003668045.1_CriGri-PICR_genomic.fna -checkGC             -single /data/isshamie/TSS/NCBI_PICR_103_Processed/external/Sequencing/PRJEB9291/processed/sortBam/trim_ERS2502088_1.bam
makeTagDirectory /data/isshamie/TSS/NCBI_PICR_103_Processed/external/Sequencing/PRJEB9291/processed/tagDir/ERS715572_1 -genome /data/isshamie/genome/ncbi_anno_103/GCF_003668045.1_CriGri-PICR_genomic.fna -checkGC             -single /data/isshamie/TSS/NCBI_PICR_103_Processed/external/Sequencing/PRJEB9291/processed/sortBam/trim_ERS715572_1.bam
makeTagDirectory /data/isshamie/TSS/NCBI_PICR_103_Processed/external/Sequencing/PRJEB9291/processed/tagDir/ERS2502102_1 -genome /data/isshamie/genome/ncbi_anno_103/GCF_003668045.1_CriGri-PICR_genomic.fna -checkGC             -single /data/isshamie/TSS/NCBI_PICR_103_Processed/external/Sequencing/PRJEB9291/proc

## Make bedgraph files

In [8]:
for curr_tag in glob.glob(tag_dir + "/*"): 
    curr_name = os.path.join(bedgraph_dir, os.path.basename(curr_tag))
    cmd = "makeUCSCfile {curr_tag} -o {curr_name}".format(curr_tag=curr_tag, curr_name=curr_name)
    print(cmd)
    os.system(cmd)

makeUCSCfile /data/isshamie/TSS/NCBI_PICR_103_Processed/external/Sequencing/PRJEB9291/processed/tagDir/ERS2502089_1 -o /data/isshamie/TSS/NCBI_PICR_103_Processed/external/Sequencing/PRJEB9291/processed/bedgraphs/ERS2502089_1
makeUCSCfile /data/isshamie/TSS/NCBI_PICR_103_Processed/external/Sequencing/PRJEB9291/processed/tagDir/ERS2502088_1 -o /data/isshamie/TSS/NCBI_PICR_103_Processed/external/Sequencing/PRJEB9291/processed/bedgraphs/ERS2502088_1
makeUCSCfile /data/isshamie/TSS/NCBI_PICR_103_Processed/external/Sequencing/PRJEB9291/processed/tagDir/ERS2502103_1 -o /data/isshamie/TSS/NCBI_PICR_103_Processed/external/Sequencing/PRJEB9291/processed/bedgraphs/ERS2502103_1
makeUCSCfile /data/isshamie/TSS/NCBI_PICR_103_Processed/external/Sequencing/PRJEB9291/processed/tagDir/ERS2502075_1 -o /data/isshamie/TSS/NCBI_PICR_103_Processed/external/Sequencing/PRJEB9291/processed/bedgraphs/ERS2502075_1
makeUCSCfile /data/isshamie/TSS/NCBI_PICR_103_Processed/external/Sequencing/PRJEB9291/processed/tagD

## findPeaks

In [5]:
inputs = " ".join(design[design["group"] == "Input"]["fastq_1"].apply(lambda x: x.split(".fastq.gz")[0]))

for ind, val in design[design["group"] != "Input"].groupby("group"):
    print(val)
    tags = " ".join(val["fastq_1"].apply(lambda x: os.path.join(tag_dir, x.split(".fastq.gz")[0])))
    
    curr_out = os.path.join(peak_dir, ind)
    cmd = "getDifferentialPeaksReplicates.pl -t {tags}  -i {inputs} > {curr_out}".format(tags=tags, inputs=inputs,
                                                                                        curr_out=curr_out)
    print(cmd)
    os.system(cmd)
    #cmd = "findPeaks <tag directory> -style histone -o auto -i <input tag directory>"

    group  replicate             fastq_1 fastq_2  antibody control
9  #Input          3  ERS715567.fastq.gz     NaN       NaN     NaN
getDifferentialPeaksReplicates.pl -t /data/isshamie/TSS/NCBI_PICR_103_Processed/external/Sequencing/PRJEB9291/processed/tagDir/ERS715567  -i ERS715556 ERS715572 ERR868163 > /data/isshamie/TSS/NCBI_PICR_103_Processed/external/Sequencing/PRJEB9291/processed/peaksDir/#Input
     group  replicate              fastq_1 fastq_2  antibody control
0  H3K27Ac          1  ERS2502089.fastq.gz      Ac     Input     NaN
1  H3K27Ac          2  ERS2502088.fastq.gz      Ac    Input      NaN
getDifferentialPeaksReplicates.pl -t /data/isshamie/TSS/NCBI_PICR_103_Processed/external/Sequencing/PRJEB9291/processed/tagDir/ERS2502089 /data/isshamie/TSS/NCBI_PICR_103_Processed/external/Sequencing/PRJEB9291/processed/tagDir/ERS2502088  -i ERS715556 ERS715572 ERR868163 > /data/isshamie/TSS/NCBI_PICR_103_Processed/external/Sequencing/PRJEB9291/processed/peaksDir/H3K27Ac
     group  

In [ ]:
inputs = " ".join(design[design["group"] == "Input"]["fastq_1"].apply(lambda x: x.split(".fastq.gz")[0]))

for ind, val in design[design["group"] != "Input"].groupby("group"):
    print(val)
    tags = " ".join(val["fastq_1"].apply(lambda x: os.path.join(tag_dir, x.split(".fastq.gz")[0])))
    
    curr_out = os.path.join(peak_dir, ind)
    cmd = "findPeaks.pl -t {tags}  -i {inputs} > {curr_out}".format(tags=tags, inputs=inputs,
                                                                                        curr_out=curr_out)
    print(cmd)
    #cmd = "findPeaks <tag directory> -style histone -o auto -i <input tag directory>"